In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xnodes=[2.2,1.1,0.7,0.4,0.25,0.1,0,-0.1,-0.25,-0.4,-0.7,-1.1,-2.2]
ynodes=[0.4322,0.7534,0.9459,0.9756,0.8,0.3846,0,-0.3846,-0.8,-0.9756,-0.9459,-0.7534,-0.4322]

#Computing h coefficients
h=[]
for i in range(len(xnodes)-1):
    h.append(xnodes[i+1]-xnodes[i])

#Computing u coefficients
u=[]
v=[]
for i in range(1,len(h)):
    u.append(2*(h[i]+h[i-1]))
    v.append(6*((ynodes[i+1]-ynodes[i])/h[i]-(ynodes[i]-ynodes[i-1])/h[i-1]))

#Setting up tridiagonal matrix by first making a diagonal matrix then add the remaining elements using symmetric property
u_diag=np.array(u)
A=np.diag(u_diag)
for i in range(len(u)-1):
    A[i][i+1]=h[i+1]
    A[i+1][i]=A[i][i+1]

#Finding z by Gaussian elimination
#Step 1: Forward elimination
for i in range(1,len(u)-1):
    elim=[]
    for j in range(len(A[i-1])):
        elim.append(A[i-1][j]*(A[i][i-1]/A[i-1][i-1]))
    A[i]=np.subtract(A[i],elim)
    v[i]=v[i]-(v[i-1]*A[i][i-1]/A[i-1][i-1])

#Step 2: Backward substitution
z=[A[10][10]]
for i in range(1,11):
    z.insert(0,(v[10-i]-A[10-i][10-i+1]*z[0])/A[10-i][10-i])

#Inserting "natural" boundary condition
z.insert(0,0)
z.append(0)

#Define a function that calculates S_i(x)
def Si(t1,t2,y1,y2,z1,z2,h,x):
    first_term=(z2*(x-t1)**3)/(6*h)
    second_term=(z1*(x-t2)**3)/(6*h)
    third_term=((y2/h)-(z2*h/6))*(x-t1)
    fourth_term=((y1/h)-(z1*h)/6)*(x-t2)
    return first_term-second_term+third_term-fourth_term

#Define the piecewise polynomials
def cubic_spline(x):
    y=0
    for i in range(len(xnodes)-1):
        if xnodes[i+1]<= x <=xnodes [i]:
            y+=Si(xnodes[i],xnodes[i+1],ynodes[i],ynodes[i+1],z[i],z[i+1],h[i],x)
    return(y)

#Polynomial interpolation using Newton's method
#Function to find Newton polynomials coefficient   
def divided_diff(xnodes,ynodes):  
    c=[]  
    for i in range(len(ynodes)):  
        c.append(ynodes[i])  
    for j in range (1,len(ynodes)):  
        for k in range (12,j-1,-1):  
            c[k]=(c[k]-c[k-1])/(xnodes[k]-xnodes[k-j])  
    return c  
  
#Evaluation of polynomials  
def newton_poly(x,c,xnodes):  
    y=0  
    for i in range(len(xnodes)):  
        p=c[i]  
        for j in range(i):  
            p=p*(x-xnodes[j])  
        y+=p  
    return(y)  

#Plotting the cubic spline
vecfunc=np.vectorize(cubic_spline)
xaxis=np.arange(-2.2,2.25,0.05)
yaxis1=vecfunc(xaxis)
plot1=plt.plot(xaxis,yaxis1)
plt.scatter(xnodes,ynodes,color='Orange')
plt.title("Natural Cubic Spline",fontsize=18)
plt.show(plot1)

#Plotting Newton polynomial interpolation
yaxis2=[]
for i in range(len(xaxis)):
    yaxis2.append(newton_poly(xaxis[i],divided_diff(xnodes,ynodes),xnodes))
plot2=plt.plot(xaxis,yaxis2,color='Black')
plt.scatter(xnodes,ynodes,color='Red')
plt.title("Polynomial Interpolation", fontsize=18)
plt.show(plot2)

#Printing parameters for natural cubic spline and newton interpolation
#For natural cubic spline, I will display only the z values (second derivative at node points), the piecewise polynomial
#can be constructed from this parameter
print("z parameters of natural cubic spline: "+str(z))
print("Newton coefficients: "+str(divided_diff(xnodes,ynodes)))